In [1]:
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
import torch
from datasets import load_dataset
from datasets.iterable_dataset import IterableDataset
from transformers import OPTForCausalLM
from chemlactica.utils.model_utils import load_model
from chemlactica.utils.utils import get_tokenizer
import scipy
import numpy as np
from sklearn.metrics import root_mean_squared_error
from rdkit import Chem

In [17]:
data = "HLM"
dataset = load_dataset(
                "/auto/home/menuab/code/sft_data/ADME_RLM"
            )

In [28]:
dataset=load_dataset("gayane/Lipo")

In [29]:
dataset

DatasetDict({
    train: Dataset({
        features: ['smiles', 'activity'],
        num_rows: 3360
    })
    validation: Dataset({
        features: ['smiles', 'activity'],
        num_rows: 420
    })
    test: Dataset({
        features: ['smiles', 'activity'],
        num_rows: 420
    })
})

In [3]:
tokenizer = get_tokenizer("/auto/home/menuab/code/ChemLactica/chemlactica/tokenizer/ChemLacticaTokenizer66")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Process 2433736 created a tokenizer


In [27]:
!ls "/nfs/dgx/raid/chem/checkpoints/facebook/galactica-125m/f10e080952974379b0e99bad"

checkpoint-106	checkpoint-265	checkpoint-424	checkpoint-530	checkpoint-689
checkpoint-159	checkpoint-318	checkpoint-477	checkpoint-583	checkpoint-742
checkpoint-212	checkpoint-371	checkpoint-53	checkpoint-636	checkpoint-795


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
model.device

device(type='cuda', index=0)

In [35]:
model_path_newPT = "/nfs/dgx/raid/chem/checkpoints/facebook/galactica-125m/f8caabf2b00f4662ae55939a/checkpoint-2000/"
model_path_newSFT = "/nfs/dgx/raid/chem/checkpoints/facebook/galactica-125m/f10e080952974379b0e99bad/checkpoint-795/"
model = load_model(model_path_newSFT, use_flash_attn=True, gradient_checkpointing=False, dtype=torch.float16).to('cuda').eval()
# model = OPTForCausalLM.from_pretrained(model_path_newSFT, attn_implementation="flash_attention_2", torch_dtype=torch.float16).to('cuda').eval()
model.device, model.dtype

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


(device(type='cuda', index=0), torch.float16)

In [34]:
prompt = tokenizer("[START_SMILES]Clc1cnc(NCC2(CN3CCCC3)CC2)nc1[END_SMILES]", return_tensors="pt").to(model.device)
out = model.generate(prompt.input_ids, do_sample=False, max_length=100)
out = tokenizer.batch_decode(out)[0]
out

'[START_SMILES]Clc1cnc(NCC2(CN3CCCC3)CC2)nc1[END_SMILES][IUPAC]5-chloro-N-[[1-(pyrrolidin-1-ylmethyl)cyclopropyl]methyl]pyrimidin-2-amine[/IUPAC][NOCOUNT]4[/NOCOUNT][NUMROTATABLEBONDS]5[/NUMROTATABLEBONDS][NUMHETEROATOMS]5[/NUMHETEROATOMS][NUMALIPHATICRINGS]2[/NUMALIPHATICRINGS][NHOHCOUNT]1[/NHOHCOUNT][NUMSATURATEDCARBOCYCLES]1[/NUMSATURATEDCARBOCYCLES][TPSA]41.05[/TPSA][QED]0.87[/QED][RINGCOUNT]3[/RINGCOUNT][NUMSATURATEDRINGS]2[/NUMSATURATEDRINGS][HEAVYATOMCOUNT]17[/HEAVYATOMCOUNT][NUMSATURATEDHETEROCYCLES]1[/NUMSATURATEDHETEROCYCLES][NUMHACCEPTORS]4[/NUMHACCEPTORS][NUMAROMATICHETEROCYCLES]'

In [36]:
ground_truths, gens, diffs = [],[],[]
invalids = 0
for sample in dataset['test']:
    ground_truth = round(sample['activity'], 2)
    prompt = f"[START_SMILES]{sample['smiles']}[END_SMILES][PROPERTY]activity "
    len_prompt = len(prompt)
    prompt = tokenizer(prompt, return_tensors="pt").to(model.device)
    out = model.generate(prompt.input_ids, do_sample=False, max_length=100)
    out = tokenizer.batch_decode(out)[0]
    try:
        gen = float(out[out.find("activity ") + len("activity "):out.find("[/PROPERTY]")])
        diff = abs(ground_truth - gen)
        print("GT:", ground_truth, "Gen:", gen, "diff:", round(diff,2), out )
        ground_truths.append(ground_truth)
        gens.append(gen)
        diffs.append(diff)
    except:
        invalids += 1
        pass

GT: 0.44 Gen: 1.23 diff: 0.79 [START_SMILES]Cc1ccccc1NC(=O)CCS(=O)(=O)c2ccc(Br)s2[END_SMILES][PROPERTY]activity 1.23[/PROPERTY][PROPERTY]Other MS 184.0431 1.23[/PROPERTY][PROPERTY]Other MS 186.0431 1.23[/PROPERTY][PROPERTY]Other MS 185.0431 1.23[/PROPERTY][PROPERTY]Other MS 186
GT: -0.81 Gen: -0.98 diff: 0.17 [START_SMILES]COCCNC(=O)c1cccc(CN2NC(=O)C3=C(C2=O)C(=O)c4ccc(Cl)cc4N3)c1[END_SMILES][PROPERTY]activity -0.98[/PROPERTY][PROPERTY]-0.89[/PROPERTY][PROPERTY]-0.89[/PROPERTY][PROPERTY]-0.89[/PROPERTY][PROPERTY]-0.89[/PROPERTY][PROPERTY]-0.89[/PROPERTY][PROPERTY]-0.89[/PROPERTY][PROPERTY]
GT: 0.48 Gen: 0.52 diff: 0.04 [START_SMILES]OC(=O)c1[nH]c2ccccc2c1CCCOc3cccc4ccccc34[END_SMILES][PROPERTY]activity 0.52[/PROPERTY][PROPERTY]Other MS 195.0161 100[/PROPERTY][PROPERTY]Other MS 287.2369 15.31[/PROPERTY][PROPERTY]Other MS 509.2772 5.31[/PROPERTY][PROPERTY]Other MS 255.
GT: 1.43 Gen: 1.1 diff: 0.33 [START_SMILES]COc1ccc2nc(C)cc(OCC(=O)Nc3ccccc3OC)c2c1[END_SMILES][PROPERTY]activity 1.1[/PR

/auto/home/menuab/miniforge3/envs/cl11.8_t_4.37/lib/python3.10/site-packages/transformers/generation/utils.py:1136: UserWarning: Input length of input_ids is 126, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


GT: -1.14 Gen: -0.98 diff: 0.16 [START_SMILES]C1CN2C[C@@H](N=C2S1)c3ccccc3[END_SMILES][PROPERTY]activity -0.98[/PROPERTY][PROPERTY]Chemical Classes Other Classes -1.48[/PROPERTY][PROPERTY]GHS Classification Warning[/PROPERTY][PROPERTY]GHS Classification H315 (100[/PROPERTY][PROPERTY]GHS Classification H319 (100[/PROPERTY][PROPERTY]Other MS 111.01[/PROPERTY][PROPERTY]Other MS 113.
GT: 1.02 Gen: 1.28 diff: 0.26 [START_SMILES]N\C(=C(/C#N)\c1ccccc1C(F)(F)F)\Sc2ccc(N)cc2[END_SMILES][PROPERTY]activity 1.28[/PROPERTY][PROPERTY]GC-MS m/z Top Peak: 191[/PROPERTY][PROPERTY]GC-MS m/z 2.7[/PROPERTY][PROPERTY]GC-MS m/z 3rd Highest: 192[/PROPERTY][NUMHDONORS]2[/NUMHDONORS][NUMSATURATEDHETEROCYCLES]0[/NUMSATURATEDHETEROCYCLES][TPSA]75
GT: 1.1 Gen: 0.35 diff: 0.75 [START_SMILES]Cc1oc(nc1CCOc2ccc(C[C@H](Nc3ccccc3C(=O)c4ccccc4)C(=O)O)cc2)c5ccccc5[END_SMILES][PROPERTY]activity 0.35[/PROPERTY][PROPERTY]Other MS 11.03[/PROPERTY][PROPERTY]Other MS 10.02[/PROPERTY][PROPERTY]Other MS 7.01[/PROPERTY][PROPERTY]

/auto/home/menuab/miniforge3/envs/cl11.8_t_4.37/lib/python3.10/site-packages/transformers/generation/utils.py:1136: UserWarning: Input length of input_ids is 104, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


GT: -2.36 Gen: -1.48 diff: 0.88 [START_SMILES]CN1[C@@H]2CC[C@H]1C[C@H](C2)OC(=O)C(CO)c3ccccc3[END_SMILES][PROPERTY]activity -1.48[/PROPERTY][PROPERTY]GC-MS m/z Top Peak: -1.2[/PROPERTY][PROPERTY]GC-MS m/z -1.2[/PROPERTY][PROPERTY]GC-MS m/z -1.2[/PROPERTY][PROPERTY]GC-MS m/z 
GT: 0.02 Gen: -0.35 diff: 0.37 [START_SMILES]CN(C)CC(C)(C)COc1nccc(Nc2cc(NC(=O)c3ccnc(c3)N4CCOCC4)ccc2C)n1[END_SMILES][PROPERTY]activity -0.35[/PROPERTY][PROPERTY]Other MS 115.07[/PROPERTY][PROPERTY]Other MS 113.065 100[/PROPERTY][PROPERTY]Other MS 97.06[/PROPERTY][PROPERTY]Other MS 9
GT: 0.87 Gen: 0.44 diff: 0.43 [START_SMILES]CCOC(=O)c1ccc(OCCC2CN(C2)c3ccc(C)nn3)cc1[END_SMILES][PROPERTY]activity 0.44[/PROPERTY][PROPERTY]Other MS 101.06[/PROPERTY][PROPERTY]Other MS 103.06[/PROPERTY][PROPERTY]Other MS 115.064 1[/PROPERTY][PROPERTY]Other MS 101.064 1[/PROPERTY][PROPERTY]Other MS 103.064
GT: 0.24 Gen: 0.26 diff: 0.02 [START_SMILES]Cc1ccc(cc1NC(=O)c2ccc(OCc3ccccn3)cc2)c4ccc(N)nn4[END_SMILES][PROPERTY]activity 0.26[/PR

/auto/home/menuab/miniforge3/envs/cl11.8_t_4.37/lib/python3.10/site-packages/transformers/generation/utils.py:1136: UserWarning: Input length of input_ids is 223, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


GT: 1.33 Gen: 0.93 diff: 0.4 [START_SMILES]CCc1c(C)nc(SCC(=O)Nc2nc(cs2)c3ccc(OC)cc3)nc1O[END_SMILES][PROPERTY]activity 0.93[/PROPERTY][PROPERTY]GC-MS m/z Top Peak: 115[/PROPERTY][PROPERTY]GC-MS m/z 2nd Highest: 116[/PROPERTY][PROPERTY]GC-MS m/z 3rd Highest: 117[/PROPERTY][NUMHDONORS]2[/NUMHDONORS][SYNONYM]Opre
GT: 0.63 Gen: -0.56 diff: 1.19 [START_SMILES]CCCCC(CC)COC(=O)CC(C(=O)OCC(CC)CCCC)S(=O)(=O)O[END_SMILES][PROPERTY]activity -0.56[/PROPERTY][PROPERTY]GHS Classification Danger[/PROPERTY][PROPERTY]GHS Classification H314 -1.88[/PROPERTY][PROPERTY]GHS Classification H318 -1.89[/PROPERTY][PROPERTY]GHS Classification H319 -1.89[/PROPERTY][PROPERTY]
GT: 0.85 Gen: -0.05 diff: 0.9 [START_SMILES]Cc1nc(nc(N2CCCC2)c1Cl)c3ccccn3[END_SMILES][PROPERTY]activity -0.05[/PROPERTY][PROPERTY]Other MS 191.1481 1.02[/PROPERTY][PROPERTY]Other MS 193.1481 1.02[/PROPERTY][PROPERTY]Other MS 61[/PROPERTY][PROPERTY]Other MS 193.1481 1.02[/PROPERTY][PROPERTY]Other MS 61
GT: 0.24 Gen: 0.85 diff: 0.61 [START_SM

In [37]:
r, p = scipy.stats.pearsonr(np.array(ground_truths), np.array(gens))
r, p

(0.8710235496802885, 3.4645847851442113e-130)

In [38]:
root_mean_squared_error(ground_truths, gens)

0.5287612780911941

In [36]:
Chem.MolToSmiles(Chem.MolFromSmiles("CN(C)C(=O)c1ccccc1"))

'CN(C)C(=O)c1ccccc1'

In [19]:
plt.scatter(ground_truths, gens, alpha=0.1)
plt.plot((0,3),(0,3))

NameError: name 'plt' is not defined

In [34]:
gens

[0.68,
 0.68,
 0.68,
 0.68,
 0.68,
 1.21,
 0.68,
 0.68,
 1.01,
 0.68,
 0.68,
 0.68,
 1.22,
 0.68,
 1.22,
 0.68,
 0.68,
 0.68,
 1.01,
 0.68,
 0.68,
 0.68,
 1.11,
 0.68,
 0.68,
 0.68,
 0.68,
 0.68,
 1.91,
 0.68,
 0.68,
 1.22,
 0.68,
 0.68,
 0.68,
 1.22,
 0.68,
 0.68,
 0.68,
 0.68,
 0.68,
 0.68,
 0.68,
 1.21,
 0.68,
 0.68,
 1.02,
 0.68,
 2.22,
 0.68,
 0.68,
 0.68,
 0.68,
 0.68,
 0.68,
 1.11,
 0.68,
 1.95,
 1.21,
 1.21,
 0.68,
 1.01,
 0.68,
 0.68,
 0.68,
 1.22,
 0.68,
 0.68,
 0.68,
 0.68,
 1.11,
 1.91,
 0.68,
 1.22,
 0.68,
 0.68,
 1.11,
 0.68,
 0.68,
 0.68,
 1.91,
 1.95,
 1.21,
 0.68,
 0.68,
 0.68,
 0.68,
 1.01,
 1.11,
 1.95,
 0.68,
 0.68,
 0.68,
 1.11,
 0.68,
 1.11,
 1.95,
 1.11,
 1.22,
 1.11,
 0.68,
 0.68,
 2.22,
 1.22,
 0.68,
 1.21,
 0.68,
 1.11,
 1.92,
 0.68,
 0.68,
 1.95,
 1.91,
 0.68,
 0.68,
 0.68,
 0.68,
 1.95,
 0.68,
 1.01,
 0.68,
 0.68,
 0.68,
 1.11,
 1.01,
 0.68,
 0.68,
 0.68,
 0.68,
 1.91,
 0.68,
 1.01,
 1.95,
 0.68,
 0.68,
 0.68,
 0.68,
 1.21,
 0.68,
 0.68,
 0.68,
 0.68,
 1.11,

In [35]:
ground_truths

[0.68,
 0.68,
 1.39,
 0.68,
 1.07,
 2.83,
 0.68,
 0.68,
 1.53,
 0.68,
 1.64,
 0.68,
 0.68,
 1.05,
 0.68,
 0.68,
 1.38,
 0.9,
 2.18,
 0.68,
 0.68,
 0.68,
 0.68,
 0.68,
 1.08,
 0.68,
 1.16,
 0.68,
 1.06,
 0.68,
 0.68,
 1.99,
 1.41,
 1.06,
 0.68,
 2.31,
 2.25,
 0.98,
 0.68,
 0.68,
 1.03,
 1.11,
 1.02,
 1.85,
 0.68,
 1.29,
 1.47,
 0.68,
 3.37,
 0.68,
 0.77,
 1.1,
 1.01,
 0.68,
 0.68,
 1.44,
 0.68,
 2.58,
 2.34,
 0.68,
 1.52,
 1.84,
 0.68,
 1.1,
 1.4,
 1.85,
 0.89,
 1.16,
 0.68,
 0.68,
 2.51,
 1.52,
 1.32,
 2.24,
 0.68,
 2.37,
 1.03,
 0.96,
 1.85,
 0.68,
 2.6,
 1.39,
 1.91,
 1.81,
 2.36,
 0.68,
 0.68,
 2.18,
 0.68,
 2.81,
 0.68,
 1.51,
 0.78,
 2.12,
 1.02,
 1.26,
 0.94,
 1.85,
 2.7,
 1.27,
 0.68,
 1.26,
 1.76,
 1.84,
 0.68,
 1.88,
 1.54,
 1.93,
 0.68,
 0.68,
 0.68,
 2.32,
 0.93,
 0.68,
 0.68,
 0.68,
 0.68,
 1.71,
 0.68,
 1.59,
 0.68,
 1.16,
 0.68,
 1.25,
 2.75,
 0.68,
 0.68,
 1.62,
 1.38,
 2.01,
 0.68,
 0.68,
 1.69,
 0.68,
 1.7,
 0.68,
 2.21,
 2.64,
 0.68,
 0.68,
 0.68,
 1.0,
 0.68,
 1.38,


In [1]:
np.random.normal(0, 0.1)

NameError: name 'np' is not defined